In [1]:
############# BASE: TIEMPOS_MIRKO #############
import warnings
warnings.filterwarnings("ignore")

In [2]:
############# CONEXIÓN OPERCOM #############
#import cx_Oracle
#import pandas as pd

#df_credenciales = pd.read_excel("../CREDENCIALES.xlsx")
#user = df_credenciales["USER"][0]
#password = df_credenciales["PASSWORD"][0]
#host_name = df_credenciales["HOST_NAME"][0]
#database = df_credenciales["ORACLE_DB"][0]  # OPERCOM

#def connectionOPERCOM():
    #dsn = cx_Oracle.makedsn(host_name,"1521",service_name=database)
    #connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)
    #return connection

In [5]:
############# ASIGNAR VARIABLES #############
from datetime import datetime
import pandas as pd
import os

PERIODO_REPORTE = "202404"
HORA_I_ALMUERZO ="12:00:00"
HORA_F_ALMUERZO = "14:00:00"
PLAZO_MAX_EQUIPOS = "00:45:00"
PLAZO_MAX_RR = "00:30:00"
PLAZO_MAX_ALMUERZO = "01:30:00"
HORA_I_ATENCION = "09:00:00"
HORA_F_ATENCION = "18:00:00"
HORA_SALIDA_MODELO = "09:30:00"
DEMORA_MAXIMA_POSIBLE = "02:00:00"  # Toda demora de mas de 2 horas se quitará del reporte

ruta_base ="BASE.xlsx"
ruta_reporte = "reporte-final/TA_" + PERIODO_REPORTE + ".xlsx"

df = pd.read_excel(ruta_base)

In [ ]:
############# LIMPIEZA, TRANFORMACIÓN Y GENERACIÓN DE REPORTE #############
columnas_a_conservar = [
    "DS_Deudor", "VF_Estado", "Creado", "DS_Distribuidor", "CS_TipPedido", "CS_NumPretups", 
    "CS_Observaciones", "GP_Fecha", "GP_Aprob", "GP_NumPedido", "GP_Observaciones", "CC_Aprob", 
    "CC_Observaciones", "CC_Login", "CC_FechaPrimera", "CC_FechaUltima", "CS_ControlObs", 
    "CS_FechaRespObs","NPS",]
df_filtrado = df[columnas_a_conservar]

# Eliminar filas con valores nulos en la columna 'CS_TipPedido'
df_filtrado = df_filtrado.dropna(subset=["CS_TipPedido"])

# Eliminar filas donde 'CS_TipPedido' sea 'Equipos' y 'GP_Aprob' sea 'NO'
df_filtrado = df_filtrado[~((df_filtrado["CS_TipPedido"] == "Equipos") & 
                            (df_filtrado["GP_Aprob"] == "NO"))]

# Filtrar filas basándose en la columna 'CC_Login'
valores_permitidos = ["C13631", "C12973", "C10203", "C15749", "C13923", "C17126"]
df_filtrado = df_filtrado[df_filtrado["CC_Login"].isin(valores_permitidos)]

# UNIR EN UN SOLO ID_PEDIDO CS_NumPretups y GP_NumPedido
def unificar_idpedido(row):
    if row["CS_TipPedido"] == "Equipos":
        return row["GP_NumPedido"]
    else:
        return row["CS_NumPretups"]

df_filtrado["ID_Pedido"] = df_filtrado.apply(unificar_idpedido, axis=1)

# CONVERTIR A DATAIME LA COLUMNA GP_FECHA QUE NO TIENE SEGUNDOS
def convert_to_datetime_with_seconds(value):
    if pd.isnull(value):
        return value
    else:
        return pd.to_datetime(value + ":00", format="%d/%m/%Y %H:%M:%S")

df_filtrado["GP_Fecha"] = df_filtrado["GP_Fecha"].apply(convert_to_datetime_with_seconds)

# Dividir el dataframe según la columna 'CS_ControlObs'
grupo_observada = df_filtrado[df_filtrado["CS_ControlObs"] == "Observada"]
grupo_RespuestaObs = df_filtrado[df_filtrado["CS_ControlObs"] == "Observada"]
grupo_no_observada = df_filtrado[df_filtrado["CS_ControlObs"] == "No observada"]
grupo_RespuestaObs = grupo_RespuestaObs.dropna(subset=["CS_FechaRespObs"])

columnas_a_conservar_grupo_RespuestaObs = [
    "DS_Deudor", "DS_Distribuidor", "CS_TipPedido", "GP_Fecha", "ID_Pedido", "CC_Login",
    "CC_FechaUltima", "CS_FechaRespObs", "CS_ControlObs", "NPS",]
grupo_RespuestaObs = grupo_RespuestaObs[columnas_a_conservar_grupo_RespuestaObs]
columnas_a_conservar_grupo_observada = [
    "DS_Deudor", "Creado", "DS_Distribuidor", "CS_TipPedido", "ID_Pedido", 
    "GP_Fecha", "CC_Login", "CC_FechaPrimera", "CS_ControlObs", "NPS",]
grupo_observada = grupo_observada[columnas_a_conservar_grupo_observada]
columnas_a_conservar_grupo_no_observada = [
    "DS_Deudor", "Creado", "DS_Distribuidor", "GP_Fecha", "CS_TipPedido", 
    "ID_Pedido", "CC_Login", "CC_FechaPrimera", "CS_ControlObs", "NPS",]
grupo_no_observada = grupo_no_observada[columnas_a_conservar_grupo_no_observada]

# Renombrar las columnas de cada grupo
encabezado_no_observada = {
    "DS_Distribuidor":"De", "ID_Pedido":"Asunto", "CC_FechaPrimera":"Respuesta", 
    "DS_Deudor":"CODIGO", "CS_TipPedido":"PRODUCTO", "CC_Login":"Analista",
    "CS_ControlObs":"estado obsevado",}
encabezado_observada = {
    "DS_Distribuidor":"De", "ID_Pedido":"Asunto", "CC_FechaPrimera":"Respuesta", 
    "DS_Deudor":"CODIGO", "CS_TipPedido":"PRODUCTO", "CC_Login":"Analista", 
    "CS_ControlObs":"estado obsevado",}
encabezado_RespuestaObs = {
    "DS_Distribuidor":"De", "ID_Pedido":"Asunto", "CS_FechaRespObs":"Recibido", 
    "CC_FechaUltima":"Respuesta", "DS_Deudor":"CODIGO", "CS_TipPedido":"PRODUCTO", 
    "CC_Login":"Analista", "CS_ControlObs":"estado obsevado",}
grupo_no_observada = grupo_no_observada.rename(columns=encabezado_no_observada)
grupo_RespuestaObs = grupo_RespuestaObs.rename(columns=encabezado_RespuestaObs)
grupo_observada = grupo_observada.rename(columns=encabezado_observada)

# Lógica para definir el valor de 'Recibido'
def obtener_recibido(row):
    if row["PRODUCTO"] == "Equipos":
        return row["GP_Fecha"]
    else:
        return row["Creado"]

grupo_no_observada["Recibido"] = grupo_no_observada.apply(obtener_recibido, axis=1)
grupo_observada["Recibido"] = grupo_observada.apply(obtener_recibido, axis=1)

# Obtener la fecha en formato dd/mm/aaaa del campo 'Respuesta'
grupo_no_observada["Fecha"] = grupo_no_observada["Respuesta"].apply(lambda x: x.strftime("%d/%m/%Y"))
grupo_observada["Fecha"] = grupo_observada["Respuesta"].apply(lambda x: x.strftime("%d/%m/%Y"))
grupo_RespuestaObs["Fecha"] = grupo_RespuestaObs["Respuesta"].apply(lambda x: x.strftime("%d/%m/%Y"))

# Reordenar las columnas según el encabezado unico
encabezado_unico = ["Fecha", "De", "Asunto", "Recibido", "Respuesta", "CODIGO", 
                    "PRODUCTO", "Analista", "estado obsevado", "NPS",]

grupo_no_observada = grupo_no_observada[encabezado_unico]
grupo_observada = grupo_observada[encabezado_unico]
grupo_RespuestaObs = grupo_RespuestaObs[encabezado_unico]
df_final = pd.concat([grupo_observada, grupo_no_observada, grupo_RespuestaObs], ignore_index=True)

# Agregar columnas codigo analista, estado de atencion () y demora
df_final["Codigo analista"] = ""
df_final["Estado"] = ""
df_final["Demora"] = ""
df_final["Respuesta"] = pd.to_datetime(df_final["Respuesta"])

# Extraer la hora de la columna 'hora'
df_final["Recibido"] = df_final["Recibido"].dt.strftime("%H:%M:%S")
df_final["Respuesta"] = df_final["Respuesta"].dt.strftime("%H:%M:%S")

# Corregir valores de TIPO PRODUCTO y ANALISTA
mapping_producto = {"Recaudación":"RECAUDO", "Recarga Virtual":"RECARGA", "Equipos":"EQUIPOS",}
mapping_analista = {"C13631":"JUAN CARLOS HUATAY", "C10203":"JOSE LUIS VALVERDE", 
                    "C12973":"RAQUEL CAYETANO", "C15749":"DIEGO RODRIGUEZ", 
                    "C13923":"YOLANDA OLIVA", "C17126":"HENRY DIAZ",}

df_final["Codigo analista"] = df_final["Analista"]

# Reemplazar valores en la columna 'TIPO' y Analista según el mapeo
df_final["PRODUCTO"] = df_final["PRODUCTO"].replace(mapping_producto)
df_final["Analista"] = df_final["Analista"].replace(mapping_analista)

#### AJUSTES Y CALCULO DE DEMORA
def ajuste_pedidos_antes_9am(value):
    if pd.isnull(value):
        return value
    else:
        if int(value[0:2:]) < 9:
            value = HORA_I_ATENCION
        return value

df_final["Recibido"] = df_final["Recibido"].apply(ajuste_pedidos_antes_9am)

def ajuste_pedidos_antes_9am_respuesta(value):
    if pd.isnull(value):
        return value
    else:
        if int(value[0:2:]) < 9:
            value = HORA_I_ATENCION
        return value

df_final["Respuesta"] = df_final["Respuesta"].apply(ajuste_pedidos_antes_9am)

# SI LA HORA ES MAYOR QUE HF (6 PM Y  9AM) ASIGNAR LA HI COMO 9AM (HI >5 AND HF <12) ----> EL RESTO BORRAR
def ajuste_pedidos_despues_6pm(row):
    # Si el analista responde a un correo despues de las 6 pm se considera el tiempo tal cual
    recibido = row["Recibido"]
    respuesta = row["Respuesta"]
    
    if int(recibido[0:2:]) * 60 + int(recibido[3:5:]) > 18 * 60:
        if int(recibido[0:2:]) > int(respuesta[0:2:]):
            return HORA_I_ATENCION
    return recibido

df_final["Recibido"] = df_final.apply(ajuste_pedidos_despues_6pm, axis=1)

# CASO : LAS RESPUETAS ANTES DE  LA HORA DE SALIDA DEL MODELO GENERA QUE HAYA DIFERENCIAS NEGATIVAS
def ajuste_hora_salida_modelo(row):
    value = row["Recibido"]
    respuesta = row["Respuesta"]
    
    if pd.isnull(value):
        return value
    else:
        if (row["PRODUCTO"] == "EQUIPOS"
            and int(row["Recibido"][0:2:]) == int(HORA_SALIDA_MODELO[0:2:])
            and int(row["Recibido"][3:5:]) < int(HORA_SALIDA_MODELO[3:5:])):
            if (int(row["Respuesta"][0:2:]) == int(HORA_SALIDA_MODELO[0:2:])
                and int(row["Respuesta"][3:5:]) < int(HORA_SALIDA_MODELO[3:5:])):
                value
            else:
                value = HORA_SALIDA_MODELO
        return value

df_final["Recibido"] = df_final.apply(ajuste_hora_salida_modelo, axis=1)

def calculo_demora(row):
    recibido = datetime.strptime(row["Recibido"], "%H:%M:%S")
    respuesta = datetime.strptime(row["Respuesta"], "%H:%M:%S")
    time_interval = respuesta - recibido
    try:
        demora = str(datetime.strptime(str(time_interval), "%H:%M:%S"))[11::]
        return demora
    except:
        return "Error"

df_final["Demora"] = df_final.apply(calculo_demora, axis=1)

def set_estado(row):
    demora = row["Demora"]
    
    if demora == "Error":
        return "Error"
    
    demora_minutos = int(demora[0:2:]) * 60 + int(demora[3:5:])
    plazo_equipos = int(PLAZO_MAX_EQUIPOS[0:2:]) * 60 + int(PLAZO_MAX_EQUIPOS[3:5:])
    plazo_rr = int(PLAZO_MAX_RR[0:2:]) * 60 + int(PLAZO_MAX_RR[3:5:])
    
    if row["PRODUCTO"] == "EQUIPOS":
        plazo = plazo_equipos
    else:
        plazo = plazo_rr
    
    if demora_minutos > plazo:
        return "FUERA PLAZO"
    
    return "DENTRO PLAZO"

df_final["Estado"] = df_final.apply(set_estado, axis=1)

# HORA DE ALMUERZO
def ajuste_horario_almuerzo(row):
    if row["Demora"] == "Error":
        return "Error"
    
    value = row["Recibido"]
    demora = int(row["Demora"][0:2:]) * 60 + int(row["Demora"][3:5:])
    
    if (int(value[0:2:]) >= 12 and int(value[0:2:]) < 14 
        and demora < (int(PLAZO_MAX_ALMUERZO[0:2]) * 60 + int(PLAZO_MAX_ALMUERZO[3:5]) + 1)):
        return "DENTRO PLAZO"
    
    return row["Estado"]

df_final["Estado"] = df_final.apply(ajuste_horario_almuerzo, axis=1)

# ELIMINAR FUERA DE FECHA OBTENER DESDE OPERCOM LA ULTIMA FECHA QUE HAY DATOS
connection = connectionOPERCOM()
cursor = connection.cursor()
sql_SELECT_FECHA_ULTIMA_REPORTE = (
    """
    SELECT FECHA 
    FROM (SELECT DISTINCT FECHA 
        FROM USRCCD.CONSOLIDADO_TIEMPOS_ATENCION_CCD 
        WHERE PERIODO ='"""+PERIODO_REPORTE+"""'
        ORDER BY FECHA DESC) 
    WHERE ROWNUM=1
    """)
cursor.execute(sql_SELECT_FECHA_ULTIMA_REPORTE)
resultados = cursor.fetchall()

if len(resultados) == 0:
    FECHA_ULTIMA_REPORTE = "00/" + PERIODO_REPORTE[4::] + "/" + PERIODO_REPORTE[0:4:]
    print("NO HAY REGISTROS DE ESTE MES")
else:
    FECHA_ULTIMA_REPORTE = str(resultados[0][0])[0:9:]
    print("Fecha de ultimo resgisto : " + str(FECHA_ULTIMA_REPORTE))

cursor.close()
connection.close()

# FECHA_ULTIMA_REPORTE= "01/09/2023"  #hace un filtro por periodo actual
FECHA_HOY = datetime.today().strftime("%d/%m/%Y")

def fuera_fecha(value):
    mes_reporte = int(PERIODO_REPORTE[4::])
    mes = int(value[3:5])
    dia = int(value[0:2])
    dia_hoy = int(FECHA_HOY[0:2])
    mes_hoy = int(FECHA_HOY[3:5])
    dia_ultmo_reporte = int(FECHA_ULTIMA_REPORTE[0:2])
    if mes_hoy == mes_reporte:
        if mes_reporte == mes and dia < dia_hoy and dia > dia_ultmo_reporte:
            return value
    elif mes_reporte + 1 == mes_hoy and mes + 1 == mes_hoy and dia > dia_ultmo_reporte:
        return value
    return "Error"

df_final["Fecha"] = df_final["Fecha"].apply(fuera_fecha)

# Eliminar filas donde analista haya respondido un dia despues? xdddddd
df_final = df_final[~((df_final["Estado"] == "Error"))]
df_final = df_final[~((df_final["Fecha"] == "Error"))]

# ELIMINAR DEMORAS DEMASIADO GRANDES, CONFIGURAR EN VARIABLE DEMORA_MAXIMA_POSIBLE
def fuera_plazo_maximo_analista(value):
    demora = int(value[0:2]) * 60 + int(value[3:5])
    demora_maxima = int(DEMORA_MAXIMA_POSIBLE[0:2]) * 60 + int(DEMORA_MAXIMA_POSIBLE[3:5])
    
    if demora > demora_maxima:
        return "Error"
    
    return value

df_final["Demora"] = df_final["Demora"].apply(fuera_plazo_maximo_analista)
df_final = df_final[~((df_final["Demora"] == "Error"))]

# REEMPLAZAR NA EN NPS POR -1
df_final["NPS"] = df_final["NPS"].fillna(-1)
print("cantidad de registros totales : " + str(df_final.shape[0]))

In [ ]:
############# EXPORTAR Y ABRIR REPORTE #############
path = ruta_reporte
df_final.to_excel(path, index=False)
os.startfile(path)

In [3]:
############# INSERTAR EN TABLA DE OPERCOM  #############
#connection = connectionOPERCOM()
#cursor = connection.cursor()
#try:
    #connection = connectionOPERCOM()
    #cursor = connection.cursor()
        
    #for i in range(9):
    #    df_final.insert(1,'column'+str(i+1),None)

    #column_df_final = [
    #    'column1', 'CODIGO', 'De', 'Asunto',  'Analista', 'Codigo analista', 'PRODUCTO', 
    #    'column2', 'Demora', 'Estado', 'Recibido', 'Respuesta', 'Fecha', 'column3', 'column4', 
    #    'column5', 'column6', 'column7', 'column8', 'column9', 'NPS', 'estado obsevado']
    #df_final = df_final[column_df_final]
    
    #rows = [tuple(x) for x in df_final.to_numpy()]
    #rows_with_date = [row for row in rows]
    #insert_sql = f"INSERT INTO USRCCD.CONSOLIDADO_TIEMPOS_ATENCION_CCD VALUES ({','.join([':{}'.format(i+1) for i in range(len(rows_with_date[0]))])})"
    #cursor.executemany(insert_sql, rows_with_date)
    #connection.commit()
    
    #SQL_SP= """BEGIN USRCCD.ActualizaTablaTiemposAtencion; 
    #        END;"""
    #cursor.execute(SQL_SP)
    #connection.commit()
#except cx_Oracle.DatabaseError as e:
    #print("Error de base de datos:", e)
    #connection.rollback()
#finally:
    #cursor.close()
    #connection.close()